In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama3-70b-8192"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

That's an easy one!

The first person to set foot on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to walk on the moon.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.tneigroup.com/jobs/power-system-modelling-engineer-intern/")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.
















Power System Modelling Engineer - Intern - TNEI








































































Skip to content










IPSA Software




info@tneigroup.com















 
TNEI Africa





 
TNEI USA





 
TNEI Ireland




Get in touch
















 









 



About Us 






 



Leading the energy industry into a low carbon future 



TNEI is a specialist consultancy in the energy sector. We help our clients respond to the complex challenges of integrating low-carbon technology into our everyday lives.  







 
get in touch
















About 






Case Studies




Resources




TNEI Insights




Key Contacts




Careers at TNEI




Life at TNEI





















Events




News















Sectors 






 



Leading the energy industry into a low carbon future   



									TNEI operates in a wide range of sectors, covering a variety of services to the renewable and conventional energy industries. 								







 
get 

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [11]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
# Assuming json_res is a list containing a single dictionary
if isinstance(json_res, list) and len(json_res) > 0 and isinstance(json_res[0], dict):
    json_res = json_res[0]
print(type(json_res))
print(json_res)

<class 'dict'>
{'role': 'Power System Modelling Engineer – Intern', 'experience': 'Internship for 3 – 6 months', 'skills': ['Master’s degree or pursuing in Power Systems or Control Engineering with Bachelor’s degree in Electrical Engineering', 'Strong expertise in power systems modelling, analysis and simulations', 'Hands-on experience in any standard power system analysis software', 'Excellent analytical, problem-solving, and communication skills', 'Excellent coding skills in python and C++'], 'description': 'Support, development, and maintenance of a cross-platform mixed language code base, supporting multiple tools and commercial products in the fields of studying electrical power systems such as developing renewable energy dynamic models, automation scripts and algorithms to generate data driven insights from power system studies and enhance the tool to remain as front runner in paving way to net zero revolution.'}


In [12]:
type(json_res)

dict

In [13]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [14]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [15]:
job = json_res
job['skills']

['Master’s degree or pursuing in Power Systems or Control Engineering with Bachelor’s degree in Electrical Engineering',
 'Strong expertise in power systems modelling, analysis and simulations',
 'Hands-on experience in any standard power system analysis software',
 'Excellent analytical, problem-solving, and communication skills',
 'Excellent coding skills in python and C++']

In [16]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[[{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [17]:
job

{'role': 'Power System Modelling Engineer – Intern',
 'experience': 'Internship for 3 – 6 months',
 'skills': ['Master’s degree or pursuing in Power Systems or Control Engineering with Bachelor’s degree in Electrical Engineering',
  'Strong expertise in power systems modelling, analysis and simulations',
  'Hands-on experience in any standard power system analysis software',
  'Excellent analytical, problem-solving, and communication skills',
  'Excellent coding skills in python and C++'],
 'description': 'Support, development, and maintenance of a cross-platform mixed language code base, supporting multiple tools and commercial products in the fields of studying electrical power systems such as developing renewable energy dynamic models, automation scripts and algorithms to generate data driven insights from power system studies and enhance the tool to remain as front runner in paving way to net zero revolution.'}

In [18]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Aswath, a business development executive at TCS. TCS is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of TCS 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase TCS's portfolio: {link_list}
        Remember you are Aswath, BDE at TCS. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Power System Modelling Engineer – Intern Support for Your Net Zero Revolution

Dear [Hiring Manager's Name],

I came across your job posting for a Power System Modelling Engineer – Intern, and I'm excited to introduce TCS as a potential partner to support your organization's mission to pave the way for the net zero revolution.

At TCS, we have a proven track record of empowering enterprises with tailored solutions that drive scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has extensive experience in developing and maintaining complex software systems, including cross-platform mixed language code bases.

Our capabilities align perfectly with your requirements, and we're confident that we can provide the necessary support to develop renewable energy dynamic models, automation scripts, and algorithms to generate data-driven insights from power system studies. Our team is well-versed in power system analysis software